In [2]:
# importing necessary libraries
import pandas as pd
import numpy as np
from itertools import combinations

In [59]:
# load the data
data = pd.read_csv('USA_gym.csv')

top 10 score grouped by apparatus

In [60]:
# filtering data for female(w) and male(m) arhletes, grouping by Appartus, and selecting the maximum predicted score

w_data = data[data['Gender']=='w'].groupby(['Athlete', 'Apparatus'])['Score_Prediction'].max().reset_index()
m_data = data[data['Gender']=='m'].groupby(['Athlete', 'Apparatus'])['Score_Prediction'].max().reset_index()

In [61]:
# divide women's data by apparatus and sort predicted scores in descending order

w_BB = w_data[w_data['Apparatus']=='BB'].sort_values(by='Score_Prediction', ascending=False)
w_FX = w_data[w_data['Apparatus']=='FX'].sort_values(by='Score_Prediction', ascending=False)
w_UB = w_data[w_data['Apparatus']=='UB'].sort_values(by='Score_Prediction', ascending=False)
w_VT = w_data[w_data['Apparatus']=='VT'].sort_values(by='Score_Prediction', ascending=False)

In [62]:
# create a list of names for the top10 athletes in each apparatus and concatenate them

w_top10 = []

for df in [w_BB, w_FX, w_UB, w_BB]:
    df1 = df.head(10)
    top10 = df1['Athlete'].unique()
    w_top10.append(top10)

# concatenate all arrays within the list into one array
w_top10 = np.concatenate(w_top10)

# remove duplicate values
w_top10 = np.unique(w_top10)

# check the final result
print(w_top10)

['BILES Simone' 'BLAKELY Skye' 'BOOTH Charlotte' 'CAREY Jade'
 'CHILES Jordan' 'DICELLO Kayla' 'JONES Shilese' 'JONG Katelyn'
 'LEE Sunisa' 'LINCOLN Kaliya' 'MCCLAIN Konnor' 'MILLER Zoe'
 'ROBERSON Joscelyn' 'SUMANASEKERA Tiana' 'WONG Leanne']


In [63]:
m_data['Apparatus'].unique()

array(['HB', 'PB', 'PH', 'FX', 'SR', 'VT'], dtype=object)

In [64]:
# divide men's data by apparatus and sort predicted scores in descending order

m_PB = m_data[m_data['Apparatus']=='PB'].sort_values(by='Score_Prediction', ascending=False)
m_SR = m_data[m_data['Apparatus']=='SR'].sort_values(by='Score_Prediction', ascending=False)
m_FX = m_data[m_data['Apparatus']=='FX'].sort_values(by='Score_Prediction', ascending=False)
m_HB = m_data[m_data['Apparatus']=='HB'].sort_values(by='Score_Prediction', ascending=False)
m_PH = m_data[m_data['Apparatus']=='PH'].sort_values(by='Score_Prediction', ascending=False)
m_VT = m_data[m_data['Apparatus']=='VT'].sort_values(by='Score_Prediction', ascending=False)

In [65]:
# create a list of names for the top10 athletes in each apparatus and concatenate them

m_top10 = []

for df in [m_PB, m_SR, m_FX, m_HB, m_PH, m_VT]:
    df1 = df.head(10)
    top10 = df1['Athlete'].unique()
    m_top10.append(top10)

# concatenate all arrays within the list into one array
m_top10 = np.concatenate(m_top10)

# remove duplicate values
m_top10 = np.unique(m_top10)

# check the final result
print(m_top10)

['ALFONSO Javier' 'ARTLIP Michael' 'BOCK Cameron' 'BRIONES Brandon'
 'CHOU J.R.' 'CHRISTOPULOS Taylor Troy' 'DANG Brandon' 'GUIMARAES Vitaliy'
 'HONG Asher' 'HOOPES Patrick' 'JAROH Michael' 'JUDA Paul'
 'KARNES Joshua Andrew' 'LOOS Riley' 'MALONE Brody' 'MANIVONG Evan'
 'MCCOOL Connor' 'MOLDAUER Yul' 'NEDOROSCIK Stephen' 'NELSON Kameron'
 'PHILLIPS Curran' 'RICHARD Fred' 'RICHARD Frederick'
 'RICHARD Frederick Nathaniel' 'RYAN Kellen' 'SKIRKEY Ian'
 'THACKSTON Parker' 'WALKER Colt' 'WHITTENBURG Donnell' 'WISKUS Shane'
 'YOUNG Khoi']


# Team score prediction

In [66]:
# create a dataframe for the predicted scores of each athlete by apparatus

def score_df(data):
    app_lst = data["Apparatus"].unique()
    df = pd.DataFrame()
    df["Athlete"]=''
    for app in app_lst:
        df_comp = data[data["Apparatus"]==app][["Athlete", "Score_Prediction"]]
        df_comp.rename(columns={"Score_Prediction":app}, inplace=True)
        df = pd. merge(df, df_comp, on="Athlete", how = "outer" )

    return df

In [68]:
# extract data for athletes in the top10 list

top_w = w_data[w_data['Athlete'].isin(w_top10)]
top_m = m_data[m_data['Athlete'].isin(m_top10)]

In [46]:
# Create a DataFrame for predicted scores by apparatus for athletes in the top 10 list

w_df = score_df(top_w)
m_df = score_df(top_m)

# Woman

In [69]:
# create combinations of 5 athletes from w_df
Athlete_list = w_df["Athlete"].unique()
all_combinations = list(combinations(Athlete_list, 5))
n = len(all_combinations)

team_score_df= pd.DataFrame(columns=["p1", 'p2', 'p3', 'p4', 'p5', "Score"])

In [70]:
n

3003

In [71]:
#5명 고르기
w_team_score_df = team_score_df

for i in range(n):
    n_lst = all_combinations[i]

    if 'Athlete' in w_df.columns:
        team_df = w_df[w_df["Athlete"].isin(n_lst)]

   # proceed only if each apparatus has at least 3 scores for individual all-around
    for col in team_df.columns:
        result = 0
        if team_df[col].count() < 3:
            result=1

    if result == 0:
        # calculate the sum of the top3 scores for each apparatus
        top_scores = ['top_scores']
        for i in range(len(team_df.columns)-1):
            df1 = team_df.iloc[:, i+1].sort_values(ascending=False)
            top3 = df1[:3]
            top_scores.append(top3.sum())
        new = pd.DataFrame(top_scores)
        new = new.T
        new.columns = team_df.columns
        df2 = pd.concat([team_df, new])
        #
        df3 = df2[df2['Athlete']=='top_scores'].drop(columns = 'Athlete')

        # create a new row with the combination and the total score
        new_row = pd.DataFrame({"p1": n_lst[0], 'p2':n_lst[1], 'p3':n_lst[2], 'p4':n_lst[3], 'p5':n_lst[4], "Score": df3.sum(axis=1)})
        # concatenate the new row to the team_score_df
        w_team_score_df = pd.concat([new_row, w_team_score_df])

In [72]:
# sort the w_team_score_df in descending order based on the 'score' column
w_team_score_df = w_team_score_df.sort_values(by='Score', ascending=False)
w_team_score_df

p1              p2                 p3                  p4  \
0      BILES Simone      CAREY Jade      JONES Shilese          LEE Sunisa   
0      BILES Simone      CAREY Jade      DICELLO Kayla       JONES Shilese   
0      BILES Simone   DICELLO Kayla      JONES Shilese          LEE Sunisa   
0      BILES Simone   CHILES Jordan      JONES Shilese          LEE Sunisa   
0      BILES Simone      CAREY Jade      JONES Shilese          LEE Sunisa   
..              ...             ...                ...                 ...   
0   BOOTH Charlotte   CHILES Jordan  ROBERSON Joscelyn  SUMANASEKERA Tiana   
0     CHILES Jordan  MCCLAIN Konnor  ROBERSON Joscelyn  SUMANASEKERA Tiana   
0     DICELLO Kayla  MCCLAIN Konnor  ROBERSON Joscelyn  SUMANASEKERA Tiana   
0   BOOTH Charlotte      CAREY Jade  ROBERSON Joscelyn  SUMANASEKERA Tiana   
0        CAREY Jade  MCCLAIN Konnor  ROBERSON Joscelyn  SUMANASEKERA Tiana   

                p5       Score  
0   LINCOLN Kaliya  173.842777  
0       LEE Sunisa  173.551659  
0   LINCOLN Kaliya  173.512294  
0   LINCOLN Kaliya  173.507164  
0   MCCLAIN Konnor   173.48341  
..             ...         ...  
0      WONG Leanne  164.702711  
0      WONG Leanne  164.639236  
0      WONG Leanne  164.633227  
0      WONG Leanne   164.48928  
0      WONG Leanne  164.413366  

[1722 rows x 6 columns]

In [52]:
w_team_score_df.to_csv('w_team_score.csv', index=False)

# Man

In [46]:
# create combinations of 5 athletes from m_df
Athlete_list = m_df["Athlete"].unique()
all_combinations = list(combinations(Athlete_list, 5))
n = len(all_combinations)

team_score_df= pd.DataFrame(columns=["p1", 'p2', 'p3', 'p4', 'p5', "Score"])

In [48]:
# loop through all combinations
for i in range(n):
    # extract the current combination
    n_lst = all_combinations[i]

    # check if 'Athlete' is in the column of m_df
    if 'Athlete' in m_df.columns:
        # filter team_df based on the current combination
        team_df = m_df[m_df["Athlete"].isin(n_lst)]

    # proceed only if each apparatus has at least 3 scores for individual all-around
    for col in team_df.columns:
        result = 0
        if team_df[col].count() < 3:
            result=1

    if result == 0:
        # calculate the sum of the top3 scores for each apparatus
        top_scores = ['top_scores']
        for i in range(len(team_df.columns)-1):
            df1 = team_df.iloc[:, i+1].sort_values(ascending=False)
            top3 = df1[:3]
            top_scores.append(top3.sum())
        new = pd.DataFrame(top_scores)
        new = new.T
        new.columns = team_df.columns
        df2 = pd.concat([team_df, new])
        df3 = df2[df2['Athlete']=='top_scores'].drop(columns = 'Athlete')

        # create a new row with the combination and the total score
        new_row = pd.DataFrame({"p1": n_lst[0], 'p2':n_lst[1], 'p3':n_lst[2], 'p4':n_lst[3], 'p5':n_lst[4], "Score": df3.sum(axis=1)})

        # concatenate the new row to the team_score_df
        team_score_df = pd.concat([new_row, team_score_df])

In [50]:
# sort the m_team_score_df in descending order based on the 'score' column
m_team_score_df = team_score_df.sort_values(by='Score', ascending=False)
m_team_score_df

p1                           p2  \
0                 HONG Asher                 MALONE Brody   
0                 HONG Asher  RICHARD Frederick Nathaniel   
0               MALONE Brody          WHITTENBURG Donnell   
0                 LOOS Riley                 MALONE Brody   
0                 HONG Asher                 MALONE Brody   
..                       ...                          ...   
0   CHRISTOPULOS Taylor Troy               HOOPES Patrick   
0   CHRISTOPULOS Taylor Troy               HOOPES Patrick   
0   CHRISTOPULOS Taylor Troy               HOOPES Patrick   
0   CHRISTOPULOS Taylor Troy                 DANG Brandon   
0             HOOPES Patrick                 DANG Brandon   

                             p3                  p4                p5  \
0                  WISKUS Shane          YOUNG Khoi   PHILLIPS Curran   
0                  WISKUS Shane          YOUNG Khoi   PHILLIPS Curran   
0                  WISKUS Shane          YOUNG Khoi   PHILLIPS Curran   
0                  WISKUS Shane          YOUNG Khoi   PHILLIPS Curran   
0   RICHARD Frederick Nathaniel          YOUNG Khoi   PHILLIPS Curran   
..                          ...                 ...               ...   
0                  DANG Brandon  NEDOROSCIK Stephen       SKIRKEY Ian   
0                  DANG Brandon  NEDOROSCIK Stephen  THACKSTON Parker   
0                  DANG Brandon         SKIRKEY Ian  THACKSTON Parker   
0            NEDOROSCIK Stephen         SKIRKEY Ian  THACKSTON Parker   
0            NEDOROSCIK Stephen         SKIRKEY Ian  THACKSTON Parker   

         Score  
0   257.149549  
0   256.936239  
0   256.738977  
0   256.648462  
0   256.622599  
..         ...  
0    68.801557  
0    68.670976  
0    68.534765  
0    56.564053  
0    55.118162  

[166530 rows x 6 columns]

In [53]:
m_team_score_df.to_csv('m_team_score.csv', index=False)